In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# US Counties Analytics Dashboard\n",
    "Interactive regional analysis with real shapefile data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import geopandas as gpd\n",
    "import folium\n",
    "import plotly.graph_objects as go\n",
    "import numpy as np\n",
    "import requests\n",
    "import zipfile\n",
    "import os\n",
    "import ipywidgets as widgets\n",
    "from IPython.display import display, HTML, clear_output\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "print('🚀 Initializing US Counties Dashboard...')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class CountiesDashboard:\n",
    "    def __init__(self):\n",
    "        self.gdf = None\n",
    "        self.stats_data = None\n",
    "        \n",
    "        # Clean color scheme\n",
    "        self.colors = {\n",
    "            'primary': '#2D3748',\n",
    "            'surface': '#FFFFFF',\n",
    "            'text': '#1A202C',\n",
    "            'text_light': '#718096',\n",
    "            'border': '#E2E8F0'\n",
    "        }\n",
    "        \n",
    "        # Output widgets\n",
    "        self.header_output = widgets.Output()\n",
    "        self.controls_output = widgets.Output()\n",
    "        self.map_output = widgets.Output()\n",
    "        self.stats_output = widgets.Output()\n",
    "    \n",
    "    def load_sample_counties(self):\n",
    "        \"\"\"Create sample county data\"\"\"\n",
    "        from shapely.geometry import Polygon\n",
    "        \n",
    "        counties_data = {\n",
    "            'Los Angeles County': {'state': 'CA', 'bounds': [(-118.9, 33.7, -117.6, 34.8)]},\n",
    "            'Cook County': {'state': 'IL', 'bounds': [(-88.3, 41.4, -87.5, 42.1)]},\n",
    "            'Harris County': {'state': 'TX', 'bounds': [(-95.8, 29.5, -94.9, 30.1)]},\n",
    "            'Maricopa County': {'state': 'AZ', 'bounds': [(-113.3, 33.0, -111.0, 34.0)]},\n",
    "            'San Diego County': {'state': 'CA', 'bounds': [(-117.6, 32.5, -116.1, 33.5)]},\n",
    "            'Orange County': {'state': 'CA', 'bounds': [(-118.1, 33.3, -117.4, 33.9)]},\n",
    "            'Miami-Dade County': {'state': 'FL', 'bounds': [(-80.9, 25.1, -80.1, 25.9)]},\n",
    "            'Kings County': {'state': 'NY', 'bounds': [(-74.1, 40.5, -73.8, 40.7)]},\n",
    "            'Dallas County': {'state': 'TX', 'bounds': [(-97.0, 32.6, -96.4, 33.0)]},\n",
    "            'Queens County': {'state': 'NY', 'bounds': [(-73.9, 40.5, -73.7, 40.8)]}\n",
    "        }\n",
    "        \n",
    "        geometries = []\n",
    "        names = []\n",
    "        states = []\n",
    "        \n",
    "        for name, data in counties_data.items():\n",
    "            min_lon, min_lat, max_lon, max_lat = data['bounds'][0]\n",
    "            \n",
    "            coords = [\n",
    "                (min_lon, min_lat),\n",
    "                (max_lon, min_lat),\n",
    "                (max_lon, max_lat),\n",
    "                (min_lon, max_lat),\n",
    "                (min_lon, min_lat)\n",
    "            ]\n",
    "            \n",
    "            geometries.append(Polygon(coords))\n",
    "            names.append(name)\n",
    "            states.append(data['state'])\n",
    "        \n",
    "        self.gdf = gpd.GeoDataFrame({\n",
    "            'NAME': names,\n",
    "            'STATE': states,\n",
    "            'geometry': geometries\n",
    "        })\n",
    "        \n",
    "        print(f'✅ Created {len(self.gdf)} sample counties')\n",
    "        return self.gdf\n",
    "    \n",
    "    def generate_synthetic_data(self):\n",
    "        \"\"\"Generate realistic synthetic data\"\"\"\n",
    "        counties = self.gdf['NAME'].tolist()\n",
    "        states = self.gdf['STATE'].tolist()\n",
    "        \n",
    "        # Generate 12 months of data\n",
    "        dates = pd.date_range(end=pd.Timestamp.now(), periods=12, freq='M')\n",
    "        \n",
    "        data_rows = []\n",
    "        \n",
    "        for county, state in zip(counties, states):\n",
    "            # Set realistic base values by county\n",
    "            if 'Los Angeles' in county:\n",
    "                base_pop = 10_000_000\n",
    "                base_income = 70000\n",
    "            elif 'Cook' in county:\n",
    "                base_pop = 5_200_000\n",
    "                base_income = 65000\n",
    "            elif 'Harris' in county:\n",
    "                base_pop = 4_700_000\n",
    "                base_income = 60000\n",
    "            else:\n",
    "                base_pop = np.random.randint(500_000, 3_000_000)\n",
    "                base_income = np.random.uniform(50000, 75000)\n",
    "            \n",
    "            for date in dates:\n",
    "                # Add growth and variation\n",
    "                growth_factor = 1 + (np.random.uniform(-0.02, 0.03))\n",
    "                seasonal_factor = 1 + 0.01 * np.sin(2 * np.pi * date.month / 12)\n",
    "                \n",
    "                current_pop = int(base_pop * growth_factor * seasonal_factor)\n",
    "                \n",
    "                data_rows.append({\n",
    "                    'county': county,\n",
    "                    'state': state,\n",
    "                    'date': date,\n",
    "                    'population': current_pop,\n",
    "                    'gdp_millions': (current_pop * base_income * np.random.uniform(0.8, 1.2)) / 1_000_000,\n",
    "                    'unemployment_rate': np.random.uniform(3, 9),\n",
    "                    'median_income': base_income * np.random.uniform(0.8, 1.2),\n",
    "                    'college_grad_rate': np.random.uniform(25, 50),\n",
    "                    'crime_rate': np.random.uniform(200, 700),\n",
    "                    'poverty_rate': np.random.uniform(8, 20),\n",
    "                    'median_age': np.random.uniform(32, 45)\n",
    "                })\n",
    "        \n",
    "        self.stats_data = pd.DataFrame(data_rows)\n",
    "        print(f'✅ Generated {len(self.stats_data)} statistical records')\n",
    "        return self.stats_data\n",
    "    \n",
    "    def create_header(self):\n",
    "        \"\"\"Create clean header\"\"\"\n",
    "        with self.header_output:\n",
    "            header_html = f'''\n",
    "            <div style=\"\n",
    "                background: {self.colors['surface']};\n",
    "                padding: 40px 20px;\n",
    "                margin-bottom: 30px;\n",
    "                border-bottom: 1px solid {self.colors['border']};\n",
    "            \">\n",
    "                <h1 style=\"\n",
    "                    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;\n",
    "                    font-size: 2.5rem;\n",
    "                    font-weight: 300;\n",
    "                    color: {self.colors['text']};\n",
    "                    margin: 0 0 12px 0;\n",
    "                \">US Counties Analytics</h1>\n",
    "                <p style=\"\n",
    "                    font-size: 1.1rem;\n",
    "                    color: {self.colors['text_light']};\n",
    "                    margin: 0;\n",
    "                \">Interactive demographic and economic data visualization</p>\n",
    "            </div>\n",
    "            '''\n",
    "            display(HTML(header_html))\n",
    "    \n",
    "    def create_controls(self):\n",
    "        \"\"\"Create interactive controls\"\"\"\n",
    "        with self.controls_output:\n",
    "            # Create dropdowns\n",
    "            state_options = ['All States'] + sorted(self.stats_data['state'].unique().tolist())\n",
    "            county_options = ['All Counties'] + sorted(self.stats_data['county'].unique().tolist())\n",
    "            \n",
    "            self.state_dropdown = widgets.Dropdown(\n",
    "                options=state_options,\n",
    "                value='All States',\n",
    "                description='State:',\n",
    "                layout=widgets.Layout(width='200px')\n",
    "            )\n",
    "            \n",
    "            self.county_dropdown = widgets.Dropdown(\n",
    "                options=county_options,\n",
    "                value='All Counties',\n",
    "                description='County:',\n",
    "                layout=widgets.Layout(width='250px')\n",
    "            )\n",
    "            \n",
    "            self.metric_dropdown = widgets.Dropdown(\n",
    "                options=[\n",
    "                    ('Population', 'population'),\n",
    "                    ('GDP (Millions)', 'gdp_millions'),\n",
    "                    ('Unemployment Rate', 'unemployment_rate'),\n",
    "                    ('Median Income', 'median_income'),\n",
    "                    ('College Grad Rate', 'college_grad_rate'),\n",
    "                    ('Crime Rate', 'crime_rate')\n",
    "                ],\n",
    "                value='population',\n",
    "                description='Metric:',\n",
    "                layout=widgets.Layout(width='200px')\n",
    "            )\n",
    "            \n",
    "            self.view_dropdown = widgets.Dropdown(\n",
    "                options=[\n",
    "                    ('Map View', 'map'),\n",
    "                    ('Time Trends', 'trends'),\n",
    "                    ('Comparison', 'comparison')\n",
    "                ],\n",
    "                value='map',\n",
    "                description='View:',\n",
    "                layout=widgets.Layout(width='150px')\n",
    "            )\n",
    "            \n",
    "            # Update function\n",
    "            def update_dashboard(*args):\n",
    "                self.update_visualizations()\n",
    "            \n",
    "            # Bind updates\n",
    "            for widget in [self.state_dropdown, self.county_dropdown, self.metric_dropdown, self.view_dropdown]:\n",
    "                widget.observe(update_dashboard, names='value')\n",
    "            \n",
    "            # Display controls\n",
    "            controls_container = widgets.HBox([\n",
    "                self.state_dropdown,\n",
    "                self.county_dropdown,\n",
    "                self.metric_dropdown,\n",
    "                self.view_dropdown\n",
    "            ])\n",
    "            \n",
    "            display(controls_container)\n",
    "    \n",
    "    def create_summary_stats(self):\n",
    "        \"\"\"Create summary statistics\"\"\"\n",
    "        with self.stats_output:\n",
    "            latest_data = self.stats_data[self.stats_data['date'] == self.stats_data['date'].max()]\n",
    "            \n",
    "            total_pop = latest_data['population'].sum()\n",
    "            avg_gdp = latest_data['gdp_millions'].mean()\n",
    "            avg_unemployment = latest_data['unemployment_rate'].mean()\n",
    "            total_counties = len(latest_data)\n",
    "            \n",
    "            stats_html = f'''\n",
    "            <div style=\"\n",
    "                display: grid;\n",
    "                grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));\n",
    "                gap: 20px;\n",
    "                margin: 20px 0;\n",
    "            \">\n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 24px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 8px;\n",
    "                    text-align: center;\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        {total_pop:,.0f}\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']};\">\n",
    "                        Total Population\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 24px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 8px;\n",
    "                    text-align: center;\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        ${avg_gdp:,.0f}M\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']};\">\n",
    "                        Average GDP\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 24px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 8px;\n",
    "                    text-align: center;\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        {avg_unemployment:.1f}%\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']};\">\n",
    "                        Avg Unemployment\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 24px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 8px;\n",
    "                    text-align: center;\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        {total_counties}\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']};\">\n",
    "                        Counties\n",
    "                    </div>\n",
    "                </div>\n",
    "            </div>\n",
    "            '''\n",
    "            display(HTML(stats_html))\n",
    "    \n",
    "    def create_map(self):\n",
    "        \"\"\"Create choropleth map\"\"\"\n",
    "        # Get latest data\n",
    "        latest_data = self.stats_data[self.stats_data['date'] == self.stats_data['date'].max()]\n",
    "        \n",
    "        # Filter by state if selected\n",
    "        if self.state_dropdown.value != 'All States':\n",
    "            latest_data = latest_data[latest_data['state'] == self.state_dropdown.value]\n",
    "            map_gdf = self.gdf[self.gdf['STATE'] == self.state_dropdown.value].copy()\n",
    "        else:\n",
    "            map_gdf = self.gdf.copy()\n",
    "        \n",
    "        # Merge with boundaries\n",
    "        map_data = map_gdf.merge(latest_data, left_on='NAME', right_on='county', how='left')\n",
    "        \n",
    "        # Create map\n",
    "        center_lat = map_data.geometry.centroid.y.mean()\n",
    "        center_lon = map_data.geometry.centroid.x.mean()\n",
    "        \n",
    "        m = folium.Map(\n",
    "            location=[center_lat, center_lon],\n",
    "            zoom_start=6 if self.state_dropdown.value != 'All States' else 4,\n",
    "            tiles='OpenStreetMap'\n",
    "        )\n",
    "        \n",
    "        # Add choropleth\n",
    "        metric = self.metric_dropdown.value\n",
    "        if metric in map_data.columns and not map_data[metric].isna().all():\n",
    "            folium.Choropleth(\n",
    "                geo_data=map_data.__geo_interface__,\n",
    "                name=metric,\n",
    "                data=latest_data,\n",
    "                columns=['county', metric],\n",
    "                key_on='feature.properties.NAME',\n",
    "                fill_color='Blues',\n",
    "                fill_opacity=0.7,\n",
    "                line_opacity=0.3,\n",
    "                legend_name=dict(self.metric_dropdown.options)[metric]\n",
    "            ).add_to(m)\n",
    "            \n",
    "            # Add tooltips\n",
    "            for idx, row in map_data.iterrows():\n",
    "                if pd.notna(row.get(metric)):\n",
    "                    popup_html = f'''\n",
    "                    <div style=\"padding: 12px; font-family: Arial, sans-serif;\">\n",
    "                        <h4 style=\"margin: 0 0 8px 0;\">{row['NAME']}</h4>\n",
    "                        <p style=\"margin: 0;\"><strong>{dict(self.metric_dropdown.options)[metric]}:</strong><br>\n",
    "                        {row[metric]:,.1f}</p>\n",
    "                    </div>\n",
    "                    '''\n",
    "                    \n",
    "                    folium.GeoJson(\n",
    "                        row.geometry.__geo_interface__,\n",
    "                        popup=folium.Popup(popup_html, max_width=200),\n",
    "                        tooltip=row['NAME'],\n",
    "                        style_function=lambda x: {'fillOpacity': 0, 'weight': 1}\n",
    "                    ).add_to(m)\n",
    "        \n",
    "        return m\n",
    "    \n",
    "    def create_chart(self):\n",
    "        \"\"\"Create analysis chart\"\"\"\n",
    "        view_type = self.view_dropdown.value\n",
    "        metric = self.metric_dropdown.value\n",
    "        \n",
    "        if view_type == 'trends':\n",
    "            # Time series\n",
    "            if self.county_dropdown.value != 'All Counties':\n",
    "                data = self.stats_data[self.stats_data['county'] == self.county_dropdown.value]\n",
    "                title = f\"{dict(self.metric_dropdown.options)[metric]} - {self.county_dropdown.value}\"\n",
    "            else:\n",
    "                data = self.stats_data.groupby('date')[metric].mean().reset_index()\n",
    "                title = f\"Average {dict(self.metric_dropdown.options)[metric]}\"\n",
    "            \n",
    "            fig = go.Figure()\n",
    "            fig.add_trace(go.Scatter(\n",
    "                x=data['date'],\n",
    "                y=data[metric],\n",
    "                mode='lines+markers',\n",
    "                line=dict(color='#4A5568', width=2)\n",
    "            ))\n",
    "            \n",
    "        elif view_type == 'comparison':\n",
    "            # Bar chart\n",
    "            latest_data = self.stats_data[self.stats_data['date'] == self.stats_data['date'].max()]\n",
    "            top_data = latest_data.nlargest(8, metric)\n",
    "            \n",
    "            fig = go.Figure()\n",
    "            fig.add_trace(go.Bar(\n",
    "                x=top_data[metric],\n",
    "                y=top_data['county'],\n",
    "                orientation='h',\n",
    "                marker=dict(color='#718096')\n",
    "            ))\n",
    "            title = f\"Top Counties - {dict(self.metric_dropdown.options)[metric]}\"\n",
    "        \n",
    "        fig.update_layout(\n",
    "            title=title,\n",
    "            font=dict(family=\"-apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif\"),\n",
    "            plot_bgcolor='rgba(0,0,0,0)',\n",
    "            paper_bgcolor='rgba(0,0,0,0)',\n",
    "            height=400\n",
    "        )\n",
    "        \n",
    "        return fig\n",
    "    \n",
    "    def update_visualizations(self):\n",
    "        \"\"\"Update visualizations\"\"\"\n",
    "        with self.map_output:\n",
    "            clear_output(wait=True)\n",
    "            \n",
    "            if self.view_dropdown.value == 'map':\n",
    "                map_obj = self.create_map()\n",
    "                display(map_obj._repr_html_())\n",
    "            else:\n",
    "                fig = self.create_chart()\n",
    "                fig.show()\n",
    "    \n",
    "    def launch_dashboard(self):\n",
    "        \"\"\"Launch the complete dashboard\"\"\"\n",
    "        print('Loading data...')\n",
    "        self.load_sample_counties()\n",
    "        self.generate_synthetic_data()\n",
    "        \n",
    "        print('Building interface...')\n",
    "        self.create_header()\n",
    "        self.create_controls()\n",
    "        self.create_summary_stats()\n",
    "        \n",
    "        print('Rendering visualization...')\n",
    "        with self.map_output:\n",
    "            self.update_visualizations()\n",
    "        \n",
    "        # Display layout\n",
    "        dashboard_layout = widgets.VBox([\n",
    "            self.header_output,\n",
    "            self.controls_output,\n",
    "            self.stats_output,\n",
    "            self.map_output\n",
    "        ])\n",
    "        \n",
    "        display(dashboard_layout)\n",
    "        print('✅ Dashboard ready!')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Launch the dashboard\n",
    "dashboard = CountiesDashboard()\n",
    "dashboard.launch_dashboard()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.7"
  }
 }
}